In [124]:
import pandas as pd



def get_apis() :
    file_path = "./p5_reference.txt"
    df = pd.read_csv(file_path, sep="\n", header=None)
    
    data = {}
    current_header = ""
    
    for item in df.values.tolist() :
        ele = item[0].strip()
        if(ele[0].isupper()) :
            if(ele != "Foundation") :
                data[ele] = []
                current_header = ele
        else :
            data[current_header].append(ele)
            
#     df = pd.DataFrame().append(data, ignore_index=True)
    
    return data

script_path = "./sample.js"
content = ""

functions = get_apis()

with open(script_path, 'r') as f:
    content = f.readlines()

content = ''.join(content)



Environment  :  ['describe()', 'describeElement()', 'textOutput()', 'gridOutput()', 'print()', 'frameCount', 'deltaTime', 'focused', 'cursor()', 'frameRate()', 'noCursor()', 'displayWidth', 'displayHeight', 'windowWidth', 'windowHeight', 'windowResized()', 'width', 'height', 'fullscreen()', 'pixelDensity()', 'displayDensity()', 'getURL()', 'getURLPath()', 'getURLParams()']
Color  :  []
Creating & Reading  :  ['alpha()', 'blue()', 'brightness()', 'color()', 'green()', 'hue()', 'lerpColor()', 'lightness()', 'red()', 'saturation()', 'p5.Color']
Setting  :  ['background()', 'clear()', 'colorMode()', 'fill()', 'noFill()', 'noStroke()', 'stroke()', 'erase()', 'noErase()']
Shape  :  ['2D Primitives', 'arc()', 'ellipse()', 'circle()', 'line()', 'point()', 'quad()', 'rect()', 'square()', 'triangle()']
Attributes  :  ['textAlign()', 'textLeading()', 'textSize()', 'textStyle()', 'textWidth()', 'textAscent()', 'textDescent()', 'textWrap()']
Curves  :  ['bezier()', 'bezierDetail()', 'bezierPoint()'